# Explaining Model Decisions and Performance

## Overview

At a very high level, Howso Engine is about: 

- Making an accurate prediction (even with limited or sparse data!) 

- Explaining the prediction process 

- Showing key properties of the data 

In this notebook, we will be using the adult data set as an example to demonstrate some of Howso Engine’s capabilities, including cases and features which contribute to predictions, anomalies analysis, and potential improvements to the data to gain more insight into the data.  


In [1]:
import pandas as pd
from pmlb import fetch_data

from howso import engine
from howso.utilities import infer_feature_attributes
from howso.visuals import (
    plot_anomalies,
    plot_dataset,
    plot_feature_importances,
)

# Section 1: Train and Configure the Trainee

## Step 1: Load Data

We are using the Adult dataset in this recipe.

In [2]:
# Load adult data
df = fetch_data('adult', local_cache_dir="../../../data/adult")

# Specify column names
df.columns = ['age', 'workclass', 'fnlwgt', 'education',
              'education-num', 'marital-status', 'occupation',
              'relationship', 'race', 'sex', 'capital-gain',
              'capital-loss', 'hours-per-week', 'native-country', 'target']

# Sample the data for demo purpose
df = df.sample(1_000).reset_index(drop=True)

df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,64.0,0,257790.0,11,9.0,0,0,4,4,0,0.0,0.0,38.0,39,1
1,21.0,4,116489.0,15,10.0,4,10,3,4,0,0.0,0.0,40.0,39,1
2,46.0,6,182689.0,15,10.0,2,3,0,4,1,3103.0,0.0,35.0,39,0
3,62.0,4,499971.0,1,7.0,6,6,1,2,0,0.0,0.0,40.0,39,1
4,23.0,4,133061.0,11,9.0,2,12,0,4,1,0.0,0.0,80.0,39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,23.0,4,198992.0,11,9.0,4,1,3,4,0,0.0,0.0,35.0,39,1
996,58.0,1,26947.0,9,13.0,6,10,4,0,0,0.0,0.0,40.0,39,1
997,63.0,0,156799.0,11,9.0,6,0,1,4,0,0.0,0.0,4.0,39,1
998,19.0,0,264767.0,11,9.0,4,0,3,4,1,0.0,0.0,40.0,39,1


## Step 2: Define the Feature Attributes

For all Howso Engine usage, we must define feature attributes. We will also define our action and context features.

In [3]:
types = {
    "education": "nominal"
}

# Infer features attributes
features = infer_feature_attributes(df, types=types)

# Specify the context and action feature
action_features = ['target']
context_features = features.get_names(without=['target'])

## Step 3: Create the Trainee, Train, and Analyze

In [4]:
# Create the trainee with custom name
t = engine.Trainee(name='Engine - Predictions and Explanations Recipe', features=features, overwrite_existing=True)

# Train
t.train(df)

# Analyze the model
t.analyze(action_features=action_features, context_features=features.get_names(without=action_features))


## Step 4: Evaluate Trainee Accuracy

In [5]:
accuracy = t.react_aggregate(
    prediction_stats_action_feature=action_features[0],
    details = {
        "prediction_stats": True,
        "selected_prediction_stats": ["accuracy"]
        }
)['target'].iloc[0]

print("Test set prediction accuracy: {acc}".format(acc=accuracy))

Test set prediction accuracy: 0.822


# Section 2: Explain Predictions

How were the predictions made? 

Howso Engine provides detailed explanation for complete model transparency. Let's examine a subset of the explanations.


## Step 1: Inspect Feature Importance (Global)

Feature importance information provides insight into the features which are primary drivers for each prediction. This is important to understand in the context of AI bias and discrimination (ex. Sensitive attribute being the primary contribution to a prediction). 

This information is available at the global level (overall model), but can also be computed at the local level (regional model for each case).

In [23]:
robust_accuracy_contributions = t.react_aggregate(
    feature_influences_action_feature=action_features[0],
    details = {"feature_robust_accuracy_contributions": True}
).loc['feature_robust_accuracy_contributions'][action_features[0]]
plot_feature_importances(pd.DataFrame([robust_accuracy_contributions]), title="Global Mean Decrease in Accuracy (MDA)", yaxis_title="MDA")

MDA values for each feature indicate to the user which features have the most significant individual predictive power. Those with higher values are more important for the model's predictions. Furthermore, features with negative MDA values can even be considered detrimental to model accuracy. However, it's important to note that these are global values and features with negative MDA values could still be beneficial for model accuracy in some regions of the data.

## Step 2: Feature Uncertainty (Global)

Are there any noisy features? 

Howso Engine’s performance is robust against noisy feature[s], and can maintain a high level of accuracy despite noisy data. 

Part of the reason Howso Engine can maintain its level of performance despite noisy data is through the characterization of feature uncertainties (residuals). The feature residuals can be extracted for user review. 

> Note: Feature Residuals are in the same units as the original features which makes it easy to interpret. For example, the residual for the “age” feature has the unit of years as in the original data.

Feature residuals are available at the global level (overall model) and at the local level (regional model for each case).


In [6]:
global_feature_residuals = t.react_aggregate(
    prediction_stats_action_feature=action_features[0],
    details = {
        "prediction_stats": True,
        "selected_prediction_stats": ["mae"]
    }
)
global_feature_residuals = global_feature_residuals.T.rename(columns={'mae':'residuals'}).sort_values('residuals', ascending=False)

global_feature_residuals.iloc[0:10]

,residuals
fnlwgt,97482.384070
capital-gain,1388.705345
capital-loss,152.828951
age,9.476880
hours-per-week,8.040804
education-num,0.984431
occupation,0.790920
workclass,0.419647
relationship,0.402384
marital-status,0.331175


# Section 3: Inspect the Data for Anomalies

Howso Engine can be used to show interesting information pertaining to the data and model, such as anomalous cases and potential model improvements. 
 
For each prediction, Howso Engine can also extract the influential cases and boundary cases to provide an exact explanation to the prediction process. More details on what’s available can be found in the notebook “2-interpretability.ipynb”.


## Step 1: Identify Anomalous cases

Anomalous cases can exist in the data as either an outlier or inlier. Outliers are cases which are very different than other cases. Inliers are cases which are too similar to other cases and do not follow the expected distribution. An inlier could be a fraudulent case that is “too good to be true”. 



In [7]:
# Store the familiarity conviction into each case with `react_into_features`,
# this will be used to identify anomalous cases
t.analyze()
t.react_into_features(familiarity_conviction_addition=True, distance_contribution=True)
stored_convictions = t.get_cases(session=t.active_session, features=df.columns.tolist() + ['familiarity_conviction_addition','.session_training_index', '.session', 'distance_contribution'])

stored_convictions

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution
0,64.0,0,257790.0,11,9.0,0,0,4,4,0,0.0,0.0,38.0,39,1,22.912468,0,b79b737f-0646-489a-9509-21c6289d51ff,16.352668
1,21.0,4,116489.0,15,10.0,4,10,3,4,0,0.0,0.0,40.0,39,1,2.559458,1,b79b737f-0646-489a-9509-21c6289d51ff,8.992672
2,46.0,6,182689.0,15,10.0,2,3,0,4,1,3103.0,0.0,35.0,39,0,21.096643,2,b79b737f-0646-489a-9509-21c6289d51ff,14.140402
3,62.0,4,499971.0,1,7.0,6,6,1,2,0,0.0,0.0,40.0,39,1,2.194906,3,b79b737f-0646-489a-9509-21c6289d51ff,31.617975
4,23.0,4,133061.0,11,9.0,2,12,0,4,1,0.0,0.0,80.0,39,1,109.465185,4,b79b737f-0646-489a-9509-21c6289d51ff,16.140565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,23.0,4,198992.0,11,9.0,4,1,3,4,0,0.0,0.0,35.0,39,1,0.362772,995,b79b737f-0646-489a-9509-21c6289d51ff,2.085703
996,58.0,1,26947.0,9,13.0,6,10,4,0,0,0.0,0.0,40.0,39,1,6.766954,996,b79b737f-0646-489a-9509-21c6289d51ff,24.885073
997,63.0,0,156799.0,11,9.0,6,0,1,4,0,0.0,0.0,4.0,39,1,5.309528,997,b79b737f-0646-489a-9509-21c6289d51ff,25.555286
998,19.0,0,264767.0,11,9.0,4,0,3,4,1,0.0,0.0,40.0,39,1,0.963981,998,b79b737f-0646-489a-9509-21c6289d51ff,7.774527


Here we set a threshold for familarity conviction that can be used to identify cases as anomalies. In the case of this notebook, we use a threshold of 0.50. Cases with a conviction value below this threshold will be deemed anomalous. Then we split up these anomalies by the mean distance contribution with those lower distance contributions being inliers and those with larger distance contributions being outliers.

In [8]:
# Threshold to determine which cases will be deemed anomalous
convict_threshold = 0.5

# Extract the anomalous cases
low_convicts = stored_convictions[stored_convictions['familiarity_conviction_addition'] <= convict_threshold ].sort_values('familiarity_conviction_addition', ascending=True)

# Average distance contribution will be used to determine if a case is an outlier or inlier
average_dist_contribution = low_convicts['distance_contribution'].mean()

# A case with distance contribution greater than average will be tagged as outlier, and vise versa for inliers
cat = ['inlier' if d < average_dist_contribution else 'outlier' for d in low_convicts['distance_contribution']]
low_convicts['category'] = cat

## Step 2: Inspect Outliers

Let’s examine a few outlier cases. Outliers are cases which are very different than other cases.

In [9]:
# Extract the outliers cases
outliers = low_convicts[low_convicts['category'] == 'outlier'].reset_index(drop=True)
outliers

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,62.0,5,191520.0,10,16.0,2,10,0,4,1,99999.0,0.0,80.0,39,0,0.018421,960,b79b737f-0646-489a-9509-21c6289d51ff,516.583810,outlier
1,48.0,4,25468.0,12,14.0,0,4,1,4,1,99999.0,0.0,50.0,39,0,0.020867,961,b79b737f-0646-489a-9509-21c6289d51ff,462.440963,outlier
2,66.0,0,186061.0,15,10.0,6,0,4,2,0,0.0,4356.0,40.0,39,1,0.024540,485,b79b737f-0646-489a-9509-21c6289d51ff,404.258758,outlier
3,39.0,4,347491.0,1,7.0,0,3,1,4,1,0.0,1876.0,46.0,39,1,0.034093,160,b79b737f-0646-489a-9509-21c6289d51ff,307.191011,outlier
4,26.0,4,280093.0,4,3.0,2,3,0,4,1,0.0,1628.0,50.0,39,1,0.036572,605,b79b737f-0646-489a-9509-21c6289d51ff,290.072518,outlier
5,48.0,6,191277.0,10,16.0,2,10,0,4,1,0.0,1902.0,60.0,39,0,0.040780,819,b79b737f-0646-489a-9509-21c6289d51ff,264.317643,outlier
6,29.0,4,250967.0,8,11.0,2,7,0,4,1,0.0,1887.0,48.0,39,0,0.056432,338,b79b737f-0646-489a-9509-21c6289d51ff,205.819508,outlier
7,31.0,4,226443.0,14,15.0,2,10,0,4,1,0.0,1977.0,70.0,39,0,0.060463,521,b79b737f-0646-489a-9509-21c6289d51ff,180.561451,outlier
8,72.0,6,285408.0,14,15.0,2,10,0,4,1,0.0,2246.0,28.0,39,0,0.060867,603,b79b737f-0646-489a-9509-21c6289d51ff,184.475361,outlier
9,46.0,4,326857.0,12,14.0,2,12,0,4,1,0.0,2415.0,65.0,39,0,0.075262,817,b79b737f-0646-489a-9509-21c6289d51ff,156.475779,outlier


For each outlier, we will compute some of its Feature Residual Convictions to understand which feature values are the most surprising, which will help the user understand what makes each case anomalous.

In [10]:
# Get the case_feature_residual_convictions, influential_cases and boundary_cases
details = {
    'feature_full_residual_convictions_for_case': True
}

# Specify outlier cases
outliers_indices = outliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(case_indices=outliers_indices,
                  preserve_feature_values=df.columns.tolist(),
                  leave_case_out=True,
                  details=details)

In [11]:
# Extract the case feature residual convictions
case_feature_residual_convictions = pd.DataFrame(results['details']['feature_full_residual_convictions_for_case'])[df.columns.tolist()]

In [12]:
fig = plot_anomalies(outliers, case_feature_residual_convictions, title="Outliers", yaxis_title="Residual Conviction")
fig.show(width=1750, height=750)

The heat map explains the reason why each case was an outlier. The darker the shade of red, the more surprising the feature value is, which contributes to the case's status as an outlier.

## Step 3: Inspect Inliers

Let’s examine a few inlier cases. Inliers are cases which are too similar to other cases and do not follow the expected distribution. Inliers can be an indication of a fraudulent case that is “too good to be true”. 

In [13]:
# Get the inlier cases
inliers = low_convicts[low_convicts['category'] == 'inlier'].reset_index(drop=True)
inliers

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,48.0,2,189762.0,11,9.0,2,3,0,4,1,0.0,0.0,40.0,39,0,0.226545,648,b79b737f-0646-489a-9509-21c6289d51ff,0.848317,inlier
1,39.0,4,105813.0,11,9.0,2,3,0,4,1,0.0,0.0,40.0,39,1,0.240720,841,b79b737f-0646-489a-9509-21c6289d51ff,1.046380,inlier
2,39.0,4,98941.0,11,9.0,2,3,0,4,1,0.0,0.0,40.0,39,1,0.249298,167,b79b737f-0646-489a-9509-21c6289d51ff,1.101782,inlier
3,50.0,4,168539.0,11,9.0,2,3,0,4,1,0.0,0.0,40.0,39,0,0.252006,768,b79b737f-0646-489a-9509-21c6289d51ff,1.062333,inlier
4,43.0,4,200835.0,11,9.0,2,3,0,4,1,0.0,0.0,40.0,39,1,0.257010,226,b79b737f-0646-489a-9509-21c6289d51ff,1.117210,inlier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,22.0,4,163519.0,15,10.0,4,6,3,4,1,0.0,0.0,25.0,39,1,0.486875,399,b79b737f-0646-489a-9509-21c6289d51ff,3.287412,inlier
61,37.0,4,199753.0,15,10.0,2,3,0,4,1,0.0,0.0,35.0,39,0,0.488475,494,b79b737f-0646-489a-9509-21c6289d51ff,2.948650,inlier
62,20.0,4,174714.0,11,9.0,4,6,3,4,1,0.0,0.0,40.0,39,1,0.492771,946,b79b737f-0646-489a-9509-21c6289d51ff,3.131740,inlier
63,40.0,4,199689.0,11,9.0,0,8,1,4,0,0.0,0.0,40.0,39,1,0.495751,531,b79b737f-0646-489a-9509-21c6289d51ff,3.028122,inlier


Similarly to what we did with the outliers, we will use the Feature Residual Convictions to understand what feature values are unusual. In this case, we may expect to see many feature values with very high convictions, indicating that the feature value is particularly unsurprising.

In [14]:
# Specify the inlier cases
inliers_indices = inliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(case_indices=inliers_indices,
                  preserve_feature_values=df.columns.tolist(),
                  leave_case_out=True,
                  details=details)

In [15]:
# Extract the case feature residual convictions
case_feature_residual_convictions = pd.DataFrame(results['details']['feature_full_residual_convictions_for_case'])[df.columns.tolist()]

In [16]:
fig = plot_anomalies(inliers, case_feature_residual_convictions, title="Inliers", yaxis_title="Residual Conviction")
fig.show(width=1500, height=750)

The heat map explains the reason why each case was an inlier. The more unspurprising the value is. If this seems unintuitive, imagine a dataset where values are always hard to predict with an error of less than 15.0. If a case were to suddenly appear where every value was predicted perfectly, its Feature Residual Conviction values would all be extremely high (dark blue in the case of this visualization).

# Section 4: Finding Potential Improvements in the Dataset

Sparse regions of the model or under defined problems can make it difficult to make an accurate prediction. Howso Engine can be used to identify potential data, or model improvements by examining the residual conviction and density.

## Step 1: Compute Case Feature Residual Convictions

Case Feature Residual Convicion is a ratio of the local residual for each feature to the case residual. Inspecting these conviction values helps users understand what regions of their data are easier or more difficult to predict compared to the average across the dataset.

Understanding this can help users understand where more data collection may be beneficial.

In [17]:
# Identify cases for investigation
partial_train_df = stored_convictions
partial_train_cases = partial_train_df[['.session', '.session_training_index']]

In [18]:
# Residual convictions are output via the case_feature_residual_convictions explanation
details = {
    'feature_full_residual_convictions_for_case':True,
    'features': ['target'],
}

# Get the residual convictions for the specified cases
new_result = t.react(case_indices=partial_train_cases.values.tolist(),
                     leave_case_out=True,
                     preserve_feature_values=df.drop(action_features, axis=1).columns.tolist(),
                     action_features=action_features,
                     details=details)

In [19]:
# Extract residual conviction
target_residual_convictions = [ x['target'] for x in new_result['details']['feature_full_residual_convictions_for_case'] ]

# Binarize residual conviction
convict_threshold = 0.50
low_residual_conviction = [1 if x <= convict_threshold else 0 for x in target_residual_convictions]

# Density is just the inverse of distance_contribution
density = 1 / partial_train_df['distance_contribution']

# Add new features to the dataframe
partial_train_df['density'] = density
partial_train_df['target_residual_conviction'] = target_residual_convictions
partial_train_df['low_residual_conviction'] = low_residual_conviction

In [20]:
partial_train_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,...,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,density,target_residual_conviction,low_residual_conviction
0,64.0,0,257790.0,11,9.0,0,0,4,4,0,...,38.0,39,1,22.912468,0,b79b737f-0646-489a-9509-21c6289d51ff,16.352668,0.061152,0.675277,0
1,21.0,4,116489.0,15,10.0,4,10,3,4,0,...,40.0,39,1,2.559458,1,b79b737f-0646-489a-9509-21c6289d51ff,8.992672,0.111202,1.011170,0
2,46.0,6,182689.0,15,10.0,2,3,0,4,1,...,35.0,39,0,21.096643,2,b79b737f-0646-489a-9509-21c6289d51ff,14.140402,0.070719,1.382503,0
3,62.0,4,499971.0,1,7.0,6,6,1,2,0,...,40.0,39,1,2.194906,3,b79b737f-0646-489a-9509-21c6289d51ff,31.617975,0.031628,1.041722,0
4,23.0,4,133061.0,11,9.0,2,12,0,4,1,...,80.0,39,1,109.465185,4,b79b737f-0646-489a-9509-21c6289d51ff,16.140565,0.061956,1.559509,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,23.0,4,198992.0,11,9.0,4,1,3,4,0,...,35.0,39,1,0.362772,995,b79b737f-0646-489a-9509-21c6289d51ff,2.085703,0.479455,1.004965,0
996,58.0,1,26947.0,9,13.0,6,10,4,0,0,...,40.0,39,1,6.766954,996,b79b737f-0646-489a-9509-21c6289d51ff,24.885073,0.040185,1.227272,0
997,63.0,0,156799.0,11,9.0,6,0,1,4,0,...,4.0,39,1,5.309528,997,b79b737f-0646-489a-9509-21c6289d51ff,25.555286,0.039131,1.440864,0
998,19.0,0,264767.0,11,9.0,4,0,3,4,1,...,40.0,39,1,0.963981,998,b79b737f-0646-489a-9509-21c6289d51ff,7.774527,0.128625,1.005972,0


In [21]:
# Helper function to resize the data points
def get_sizes(min_size, max_size, series):
    min_value = series.min()
    max_value = series.max()

    m = (max_size - min_size) / (max_value - min_value)

    sizes = series * m + min_size
    return (sizes)

partial_train_df["density"] = get_sizes(5, 500, partial_train_df["density"])

With the Residual Convictions computed for the "target" feature across all the trained data, we can visualize the data to attempt to find the regions of the data where convictions are particularly low. 

In [22]:
plot_dataset(partial_train_df, x="age", y="education-num", size="density", hue="low_residual_conviction", alpha=0.4)

The above graph is a visualization of the data set in 2-dimensions, with the color as an indication of residual conviction and the size representing the density of the data. More specifically, the orange color represents the low conviction points (points which are very uncertain), and small size represents low density. Therefore, adding more data to the region with small, orange points can improve model performance. 


On the other hand, an orange point that is large would be an indication that this case lies in an dense region but was not predictable. Hence, this will be an indication where the problem is not well defined, or the data is missing key features.  


# Conclusion

In this recipe, we demonstrate many of the metrics that the Trainee provides which helps users understand the model's decision making process such as feature MDA and feature residuals.

Additionally, we show how different types of conviction can be used to identify inliers, outliers, and regions of the data where increased data collection or feature engineering may be appropriate.